In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [2]:
from settings import *

## Read Data

In [ ]:
con, meta = connectToDB('smeros', '', 'sciArticles')

sql = 'select body, retweet_count, publishing_date \
from document \
where doc_type = \'twitter\''

tweets = pd.read_sql(sql, con, index_col=None, coerce_float=True, params=None, parse_dates=None, columns=None, chunksize=None)


In [ ]:
def extractURLs(tweets):
    import re
    import requests
    import math

    urlRegex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    urls = pd.Series(tweets['body'].apply(lambda x: re.findall(urlRegex, x)), name="urls")
    
    def resolveURLs(urls):
        resolvedURLs = []
        for url in urls:
            print(url)
            try:
                from urllib.request import urlopen
                from urllib.parse import urlparse
                resolvedURL = urlopen(url,timeout=10).geturl()
                #resolvedURL = requests.head(url, allow_redirects=True, timeout=10)
                
                if(urlparse(url).netloc.endswith('.ly')):
                    print('not resolved: ' + url)
                    continue
                
                print ('resolved: ' + resolvedURL)
                print(urlopen(url).getcode())
                resolvedURLs.append(resolvedURL)
            except Exception as e:
                print (e)
                print('not resolved: ' + url)
                resolvedURLs.append('DeadLink')
        #return [session.head(url, allow_redirects=True).url for url in urls]
    
    urls = urls.apply(lambda x: resolveURLs(x))
    
    print(urls.shape)
    
    #Checks resolvability of URLs
    #resolvableURLs = urls.apply(pd.Series).applymap(lambda x: None if pd.isnull(x) else requests.get(x, verify=False).status_code==200)
    #resolvableURLs = pd.Series(np.logical_or.reduce(resolvableURLs, axis=1), name="hasResolvableURLs")
    #print (resolvableURLs.shape)

    tweets = pd.concat([tweets, urls], axis=1)

    return tweets

tweets1 = extractURLs(tweets)

http://t.co/yzUy11usmg
<urlopen error [Errno 8] nodename nor servname provided, or not known>
not resolved: http://t.co/yzUy11usmg
http://t.co/l8n3CniaB3
HTTP Error 403: Forbidden
not resolved: http://t.co/l8n3CniaB3
http://post.ly/Om3A
not resolved: http://post.ly/Om3A
http://t.co/cEVKs7ewVN
resolved: http://www.herbs-info.com/blog/new-research-suggests-sugar-causes-cancer/
200
http://ow.ly/19kQS2
not resolved: http://ow.ly/19kQS2
http://ht.ly/2QbqA
not resolved: http://ht.ly/2QbqA
http://shar.es/aFMVW
resolved: http://www.naturalnews.com/027931_GMO_crops_organ_damage.html
200
http://sbne.ws/r/4Xhe
resolved: http://www.smartbrief.com/s/2010/06/study-says-salt-lovers-might-taste-more-intensely
200
http://bit.ly/7nWQVj
not resolved: http://bit.ly/7nWQVj
http://bit.ly/7nWQVj
not resolved: http://bit.ly/7nWQVj
http://bit.ly/d6qMKl
not resolved: http://bit.ly/d6qMKl
http://t.co/Gjz0PkSjmm
<urlopen error timed out>
not resolved: http://t.co/Gjz0PkSjmm
http://t.co/cLPitxWizk
resolved: http:/

In [ ]:
#tweets1 = extractURLs(tweets)

print (tweets1['urls'][0:10])

#conn.setopt(pycurl.URL, "http://t.co/yzUy11usmg")
#conn.setopt(pycurl.URL, "http://t.co/cEVKs7ewVN")

In [ ]:
print (urls)